In [133]:
with open ('/Users/sairam/Downloads/ng-video-lecture-master/input.txt','r',) as f:
    text=f.read()

print('length of dataset in characters ', len(text))


length of dataset in characters  1115394


In [134]:
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [135]:
chars=sorted(list(set(text)))
vocab_size=len(chars)
print("".join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [136]:
stoi={ch:i for i , ch in enumerate(chars)}
itos={i:ch for i , ch in enumerate(chars)}
encode=lambda s: [stoi[c] for c in s ]#this is a function which applies the stoi dictionary mapping to the input string
decode=lambda l: "".join([itos[i] for i in l])

print(encode('hii there'))
print(decode(encode('hi there')))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hi there


In [137]:
import torch
data=torch.tensor(encode(text),dtype=torch.long)#text is our dataset and data is containing encoding values of the text
print(data.shape,data.dtype)
print(data[:1000])


torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [138]:
n=int(0.9*len(data))
train_data=data[:n]
validation_data=data[n:]

In [139]:
block_size=8
train_data[:block_size+1]


tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [140]:
x=train_data[:block_size]
y=train_data[1:block_size+1]
for t in range(block_size):
    context=x[:t+1]
    target=y[t]
    print(f'when the input is {context} the output is {target}')#----1


when the input is tensor([18]) the output is 47
when the input is tensor([18, 47]) the output is 56
when the input is tensor([18, 47, 56]) the output is 57
when the input is tensor([18, 47, 56, 57]) the output is 58
when the input is tensor([18, 47, 56, 57, 58]) the output is 1
when the input is tensor([18, 47, 56, 57, 58,  1]) the output is 15
when the input is tensor([18, 47, 56, 57, 58,  1, 15]) the output is 47
when the input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the output is 58


In [141]:
train_data.shape

torch.Size([1003854])

In [142]:
torch.manual_seed(1337)
batch_size=4# how many independent sequences will we process in parallel
block_size=8# the maximum context length for prediction ,when we train the transformer we randomly pick setences which has a length of 8

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else validation_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) #the random integers generated by this line of code will be in the range [0, 1,115,386)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])#both of these will extract 8 numbers after that like we did above in the data and stack it making both a [4,8] tensor 
    # x, y = x.to(device), y.to(device)
    return x, y

xb,yb=get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size):
    for t in range(block_size):
        context=xb[b,:t+1]
        target=yb[b,t]
        print(f"when input is {context.tolist()} the target : {target}")

# so the size of xb will be[4,8] and each row contains 8 examples as we saw above in 1 so when we feed this tensor into
# transformer we should get 4*8=32 examples to train on
        

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target : 43
when input is [24, 43] the target : 58
when input is [24, 43, 58] the target : 5
when input is [24, 43, 58, 5] the target : 57
when input is [24, 43, 58, 5, 57] the target : 1
when input is [24, 43, 58, 5, 57, 1] the target : 46
when input is [24, 43, 58, 5, 57, 1, 46] the target : 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target : 39
when input is [44] the target : 53
when input is [44, 53] the target : 56
when input is [44, 53, 56] the target : 1
when input is [44, 53, 56, 1] the target : 58
when input is [44, 53, 56, 1, 58] the target : 46
when inpu

In [143]:
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [165]:
import torch
import torch.nn as nn
from torch.nn import functional as F 
torch.manual_seed(1337)

class BigramModel(nn.Module):
    def __init__(self, vocab_size) -> None:
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size,vocab_size)

    def forward(self,idx,targets=None):# idx is a 4*8 matrix of training data samples randomly
        logits=self.token_embedding_table(idx)
        if targets is None:# we are doing this just in case we want to extract logits as the function was earlier demanding targets , which means we will generate loss
            loss=None
        else:
            B,T,C=logits.shape
            logits=logits.view(B*T,C)#this will convert the 3d vector to 2d because when we use pytorch crossentropy it expects the input to be in the form minibatch*C
            targets=targets.view(B*T)# so it will be 32*65 and targets will be 32*1, what this signifies is we are having 65 predictions to come after 32 tokens and in cross_entropy we are checkng how right we were 
            loss=F.cross_entropy(logits,targets)#comparing to targets 

        return logits,loss
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C), here we are plucking just the last character in idx, and taking its probabilities , giving it to probs and predicting next token
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
            if _ == max_new_tokens-1:
                print(logits.shape)
                print(sum(probs))
                print(idx_next)
        return idx
    
m=BigramModel(vocab_size)
logits,loss=m(xb)
print(logits.shape)
print(loss)
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 8, 65])
None
torch.Size([1, 65])
tensor([0.0485, 0.0355, 0.0065, 0.0076, 0.0213, 0.0123, 0.0023, 0.0369, 0.0203,
        0.0035, 0.0046, 0.0111, 0.0136, 0.0163, 0.0058, 0.0088, 0.0077, 0.0230,
        0.0222, 0.0417, 0.0251, 0.0200, 0.0289, 0.0092, 0.0057, 0.0067, 0.0175,
        0.0069, 0.0081, 0.0067, 0.0097, 0.0122, 0.0089, 0.0117, 0.0275, 0.0121,
        0.0077, 0.0274, 0.0026, 0.0224, 0.0019, 0.0045, 0.0129, 0.0077, 0.0080,
        0.0540, 0.0116, 0.0054, 0.0038, 0.0185, 0.0091, 0.0119, 0.0050, 0.0087,
        0.0145, 0.0048, 0.0035, 0.0370, 0.0018, 0.0071, 0.0356, 0.0191, 0.0041,
        0.0053, 0.0778], grad_fn=<AddBackward0>)
tensor([[9]])

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [145]:
optimizer=torch.optim.Adam(m.parameters(),lr=1e-3)

In [146]:
batch_size=32
for steps in range(10000):
    xb,yb=get_batch('train')
    logits,loss=m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())
    

2.572469472885132


In [147]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))

torch.Size([1, 65])
tensor([1.8251e-02, 6.9886e-02, 1.8797e-03, 4.9633e-05, 2.1558e-04, 1.5814e-04,
        6.2853e-04, 2.1214e-03, 2.0410e-04, 1.2496e-04, 1.3284e-03, 2.8734e-04,
        2.1241e-03, 3.1556e-03, 3.3588e-03, 3.2946e-03, 1.9243e-03, 2.7700e-05,
        3.0793e-03, 4.4056e-03, 3.3221e-03, 5.6676e-03, 7.9518e-04, 6.3333e-04,
        8.4944e-04, 2.9489e-03, 9.6175e-05, 2.8589e-03, 2.6557e-03, 7.5182e-04,
        2.7313e-03, 1.9270e-03, 3.0893e-02, 7.4418e-05, 1.8295e-03, 5.4527e-03,
        6.2452e-04, 2.6289e-03, 2.6361e-03, 2.4434e-03, 4.2980e-03, 1.9629e-03,
        2.3197e-01, 3.4531e-02, 2.5613e-03, 7.3800e-03, 3.0239e-03, 2.2798e-04,
        1.1707e-03, 5.4094e-04, 8.1403e-02, 3.8865e-03, 5.7273e-03, 1.3215e-03,
        2.5969e-03, 9.0154e-04, 7.7651e-03, 3.2721e-01, 8.5550e-02, 9.9976e-04,
        1.4537e-03, 2.1062e-03, 1.2316e-04, 1.1619e-03, 1.7970e-03],
       grad_fn=<AddBackward0>)
tensor([[57]])

Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith he

now from the above we got a accuracy of 2.57 using a bigram model , we were just using one character to predict the next.
now what we want is the characters to interact with each other while predicting the next token , the below code is for the same. we are averaging out the prediction of a token to come next so the thrid character will be the mean of the first two


In [176]:
torch.manual_seed(1337)
B,T,C=4,8,32
x=torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 32])

In [183]:
head_size=16
key=nn.Linear(C,head_size,bias=False)
print(f'key={key}')
query=nn.Linear(C,head_size,bias=False)
print(f'query={query}')
k=key(x)#(B,T,16) this basically means that earlier we had an embedding of 32 this will be reduced to 16 as we pass x through linear layer key so all in all each node will contain a key vector of 16 dimension 
print(f'k={k.shape}')
q=query(x)#similar to above
print(f'q={q.shape}')
wei=q@k.transpose(-2,-1)# this means we are just transposing the last two dimension T and C so this will be (B,T,16)@(B,16,T) to give output of B,T,T
# print(f'wei={wei[0]}')
trill=torch.tril(torch.ones(T,T))
# print(f'trill={trill}')
# wei=torch.zeros((T,T))
wei=wei.masked_fill(trill==0,float('-inf'))
# print(f'wei={wei[0]}')
wei=F.softmax(wei,dim=-1)
print(f'wei={wei}')
out=wei@x
        


key=Linear(in_features=32, out_features=16, bias=False)
query=Linear(in_features=32, out_features=16, bias=False)
k=torch.Size([4, 8, 16])
q=torch.Size([4, 8, 16])
wei=tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3395, 0.6605, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2869, 0.1655, 0.5475, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.3727, 0.4912, 0.1089, 0.0271, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.9357, 0.0246, 0.0092, 0.0117, 0.0188, 0.0000, 0.0000, 0.0000],
         [0.7632, 0.0689, 0.0110, 0.0082, 0.1422, 0.0065, 0.0000, 0.0000],
         [0.0107, 0.0696, 0.0848, 0.4606, 0.0324, 0.1580, 0.1839, 0.0000],
         [0.0994, 0.2195, 0.0995, 0.1397, 0.1052, 0.1152, 0.1307, 0.0908]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5242, 0.4758, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2482, 0.2221, 0.5298, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
     

In [172]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6366, 0.3634, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3060, 0.3296, 0.3643, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1979, 0.2249, 0.2577, 0.3194, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1850, 0.3748, 0.1877, 0.1959, 0.0566, 0.0000, 0.0000, 0.0000],
        [0.1087, 0.0712, 0.0772, 0.0578, 0.3750, 0.3102, 0.0000, 0.0000],
        [0.1439, 0.1283, 0.1566, 0.1666, 0.1521, 0.1075, 0.1450, 0.0000],
        [0.0922, 0.2051, 0.0634, 0.0488, 0.0449, 0.3525, 0.0959, 0.0972]],
       grad_fn=<SelectBackward0>)

above we were using bigram model , now we want the characters to interact with each other inorder to find out which character will come next. the best way to do so is to find out the average of all the character channels . we can do that by following below

In [175]:
trill=torch.tril(torch.ones(T,T))
print(trill)
wei=torch.zeros((T,T))
wei=wei.masked_fill(trill==0,float('-inf'))
print(wei)
wei=F.softmax(wei,dim=-1)
print(wei)
xbow3=wei@x#what trill does is in a matrix it makes the upper half of the triangle equal to zeros , so it is like progressively taking the average of numbers one after another execute the first two lines to understandff
torch.allclose(xbow,xbow3)


tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])
tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000,

False

In [151]:
xbow[2],xbow3[2]

(tensor([[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]]),
 tensor([[-0.6631, -0.2513],
         [ 0.1735, -0.0649],
         [ 0.1685,  0.3348],
         [-0.1621,  0.1765],
         [-0.2312, -0.0436],
         [-0.1015, -0.2855],
         [-0.2593, -0.1630],
         [-0.3015, -0.2293]]))

In [152]:
x.dtype

torch.float32

In [153]:
torch.manual_seed(42)
a=torch.tril(torch.ones(3,3))
b=torch.randint(0,10,(3,2),).float()
a=a/a.sum(1,keepdim=True)
c=a@b
print(f'a={a}')
print(f'b={b}')
print(f'c={c}')

a=tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [154]:
wei=torch.tril(torch.ones(T,T))
wei=wei/wei.sum(1,keepdim=True)
xbow2=wei@x#T,T mul B,T,C 

In [155]:
print(xbow2.shape)
print(x.shape)


torch.Size([4, 8, 2])
torch.Size([4, 8, 2])
